In [1]:
import math
import re
from   random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os

In [2]:
# Set GPU device

os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# torch.cuda.get_device_name(0)

cuda


## 1. Data

This dataset was taken from Kaggle (https://www.kaggle.com/datasets/nishantsingh96/refined-bookcorpus-dataset) for Task1

In [3]:
import pandas as pd

# Load CSV file
csv_path = "bookcorpus_80k.csv"
dataset = pd.read_csv(csv_path)

In [4]:
sentences = dataset['text'].dropna().tolist()

In [5]:
text = [x.strip().lower() for x in sentences if isinstance(x, str)] #lower case
text = [re.sub(r"[^\w\s]", '', x) for x in text]  #clean all symbols

In [6]:
for sentence in text:
    print(sentence, "_____")
    words = sentence.split()
    print(words)
    break

its probably that dammed james woman the judge thought how shed gotten into the picture was a puzzle but it probably had something to do with old busybody amy lane it seemed half the stuff that ever went on in the sister bay area had her nose poked into it _____
['its', 'probably', 'that', 'dammed', 'james', 'woman', 'the', 'judge', 'thought', 'how', 'shed', 'gotten', 'into', 'the', 'picture', 'was', 'a', 'puzzle', 'but', 'it', 'probably', 'had', 'something', 'to', 'do', 'with', 'old', 'busybody', 'amy', 'lane', 'it', 'seemed', 'half', 'the', 'stuff', 'that', 'ever', 'went', 'on', 'in', 'the', 'sister', 'bay', 'area', 'had', 'her', 'nose', 'poked', 'into', 'it']


### Making vocabs

Before making the vocabs, let's remove all question marks and perios, etc, then turn everything to lowercase, and then simply split the text. 

In [7]:
from tqdm.auto import tqdm

# Combine everything into one to make vocab
word_list = list(set(" ".join(text).split()))
word2id = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}  # special tokens

# Create the word2id in a single pass
for i, w in tqdm(enumerate(word_list), desc="Creating word2id"):
    word2id[w] = i + 4  # because 0-3 are already occupied

# Precompute the id2word mapping (this can be done once after word2id is fully populated)
id2word = {v: k for k, v in word2id.items()}
vocab_size = len(word2id)
vocab_size

/home/jupyter-st125054/.conda/envs/pytorch_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Creating word2id: 114004it [00:00, 1481063.99it/s]


114008

In [8]:
vocab_size = len(word2id)

# List of all tokens for the whole text
token_list = []

# Process sentences more efficiently
for sentence in tqdm(text, desc="Processing sentences"):
    token_list.append([word2id[word] for word in sentence.split()])

# Now token_list contains the tokenized sentences

Processing sentences: 100%|██████████| 80000/80000 [00:01<00:00, 52974.99it/s]


In [9]:
#take a look at sentences
sentences[:2]

["it's probably that dammed james woman, the judge thought. how she'd gotten into the picture was a puzzle, but it probably had something to do with old busybody amy lane. it seemed half the stuff that ever went on in the sister bay area had her nose poked into it.",
 "the great range of the tolle'fornosse'veetri, had kept the east and the west of the continent separate from the day of creation until quite recent times, recent in comparison to the true age of the world that is. for the tolle'fornosse'veetri had slowly been eroded by severe weather and movement of the continental plates, these natural forces had contributed to the reduced height of the mountains range until eventually the high mountain passes had become free from snow and ice for a few weeks each year allowing precarious passage across them. even so, these paths were not for the faint of heart. the weather was unpredictable as were natural dangers like hidden crevasses and avalanches and many wild things sought refuge i

In [10]:
#take a look at token_list
token_list[:2]

[[64632,
  59595,
  38545,
  20778,
  95621,
  21511,
  66893,
  24097,
  48879,
  113403,
  30933,
  60384,
  50098,
  66893,
  72397,
  73144,
  50205,
  8256,
  85390,
  11678,
  59595,
  17151,
  110962,
  95231,
  5460,
  18539,
  45777,
  19361,
  104864,
  6051,
  11678,
  67972,
  1861,
  66893,
  37763,
  38545,
  13436,
  77919,
  454,
  19120,
  66893,
  103747,
  20218,
  11958,
  17151,
  17210,
  72535,
  13217,
  50098,
  11678],
 [66893,
  70475,
  34316,
  80975,
  66893,
  60422,
  17151,
  49290,
  66893,
  73193,
  110853,
  66893,
  77122,
  80975,
  66893,
  28245,
  81020,
  104129,
  66893,
  43796,
  80975,
  106654,
  5036,
  21517,
  50452,
  85431,
  50452,
  19120,
  69605,
  95231,
  66893,
  51052,
  18509,
  80975,
  66893,
  82533,
  38545,
  49176,
  75040,
  66893,
  60422,
  17151,
  27259,
  29103,
  45021,
  12273,
  33675,
  34229,
  110853,
  105479,
  80975,
  66893,
  97286,
  98709,
  62778,
  11039,
  75807,
  17151,
  53159,
  95231,
  66893

In [11]:
#testing one sentence
for tokens in token_list[0]:
    print(id2word[tokens])

its
probably
that
dammed
james
woman
the
judge
thought
how
shed
gotten
into
the
picture
was
a
puzzle
but
it
probably
had
something
to
do
with
old
busybody
amy
lane
it
seemed
half
the
stuff
that
ever
went
on
in
the
sister
bay
area
had
her
nose
poked
into
it


## 2. Data loader

We gonna make dataloader.  Inside here, we need to make two types of embeddings: **token embedding** and **segment embedding**

1. **Token embedding** - Given “The cat is walking. The dog is barking”, we add [CLS] and [SEP] >> “[CLS] the cat is walking [SEP] the dog is barking”. 

2. **Segment embedding**
A segment embedding separates two sentences, i.e., [0 0 0 0 1 1 1 1 ]

3. **Masking**
As mentioned in the original paper, BERT randomly assigns masks to 15% of the sequence. In this 15%, 80% is replaced with masks, while 10% is replaced with random tokens, and the rest 10% is left as is.  Here we specified `max_pred` 

4. **Padding**
Once we mask, we will add padding. For simplicity, here we padded until some specified `max_len`. 

Note:  `positive` and `negative` are just simply counts to keep track of the batch size.  `positive` refers to two sentences that are really next to one another.

In [12]:
batch_size = 4
max_mask   = 5  # max masked tokens when 15% exceed, it will only be max_pred
max_len    = 1000 # maximum of length to be padded; 

In [13]:
def make_batch():
    batch = []
    positive = negative = 0  #count of batch size;  we want to have half batch that are positive pairs (i.e., next sentence pairs)
    while positive != batch_size/2 or negative != batch_size/2:
        
        #randomly choose two sentence so we can put [SEP]
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))
        #retrieve the two sentences
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]

        #1. token embedding - append CLS and SEP
        input_ids = [word2id['[CLS]']] + tokens_a + [word2id['[SEP]']] + tokens_b + [word2id['[SEP]']]

        #2. segment embedding - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        #3. mask language modeling
        #masked 15%, but should be at least 1 but does not exceed max_mask
        n_pred =  min(max_mask, max(1, int(round(len(input_ids) * 0.15))))
        #get the pos that excludes CLS and SEP and shuffle them
        cand_maked_pos = [i for i, token in enumerate(input_ids) if token != word2id['[CLS]'] and token != word2id['[SEP]']]
        shuffle(cand_maked_pos)
        masked_tokens, masked_pos = [], []
        #simply loop and change the input_ids to [MASK]
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)  #remember the position
            masked_tokens.append(input_ids[pos]) #remember the tokens
            #80% replace with a [MASK], but 10% will replace with a random token
            if random() < 0.1:  # 10%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = word2id[id2word[index]] # replace
            elif random() < 0.9:  # 80%
                input_ids[pos] = word2id['[MASK]'] # make mask
            else:  #10% do nothing
                pass

        # pad the input_ids and segment ids until the max len
        n_pad = max_len - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # pad the masked_tokens and masked_pos to make sure the lenth is max_mask
        if max_mask > n_pred:
            n_pad = max_mask - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        #check if first sentence is really comes before the second sentence
        #also make sure positive is exactly half the batch size
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
            
    return batch

In [14]:
batch = make_batch()

In [15]:
#len of batch
len(batch)

4

In [ ]:
#we can deconstruct using map and zip
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))
input_ids.shape, segment_ids.shape, masked_tokens.shape, masked_pos.shape, isNext.shape

## 3. Model

Recall that BERT only uses the encoder.

BERT has the following components:

- Embedding layers
- Attention Mask
- Encoder layer
- Multi-head attention
- Scaled dot product attention
- Position-wise feed-forward network
- BERT (assembling all the components)

## 3.1 Embedding

Here we simply generate the positional embedding, and sum the token embedding, positional embedding, and segment embedding together.

In [18]:
class Embedding(nn.Module):
    def __init__(self, vocab_size, max_len, n_segments, d_model, device):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(max_len, d_model)      # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)
        self.device = device

    def forward(self, x, seg):
        #x, seg: (bs, len)
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long).to(self.device)
        pos = pos.unsqueeze(0).expand_as(x)  # (len,) -> (bs, len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

## 3.2 Attention mask

In [19]:
def get_attn_pad_mask(seq_q, seq_k, device):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1).to(device)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

### Testing the attention mask

In [20]:
import torch

# Check available GPUs
print("Available GPUs:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

Available GPUs: 4
GPU 0: NVIDIA GeForce RTX 2080 Ti
GPU 1: NVIDIA GeForce RTX 2080 Ti
GPU 2: NVIDIA GeForce RTX 2080 Ti
GPU 3: NVIDIA GeForce RTX 2080 Ti


In [ ]:
print(get_attn_pad_mask(input_ids, input_ids, device).shape)

## 3.3 Encoder

The encoder has two main components: 

- Multi-head Attention
- Position-wise feed-forward network

First let's make the wrapper called `EncoderLayer`

In [22]:
class EncoderLayer(nn.Module):
    def __init__(self, n_heads, d_model, d_ff, d_k, device):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention(n_heads, d_model, d_k, device)
        self.pos_ffn       = PoswiseFeedForwardNet(d_model, d_ff)

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

Let's define the scaled dot attention, to be used inside the multihead attention

In [23]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k, device):
        super(ScaledDotProductAttention, self).__init__()
        self.scale = torch.sqrt(torch.FloatTensor([d_k])).to(device)

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / self.scale # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn 

Let's define the parameters first

In [26]:
n_layers = 6    # number of Encoder of Encoder Layer
n_heads  = 8    # number of heads in Multi-Head Attention
d_model  = 768  # Embedding Size
d_ff = 768 * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

Here is the Multiheadattention.

In [27]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, d_model, d_k, device):
        super(MultiHeadAttention, self).__init__()
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_k
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, self.d_v * n_heads)
        self.device = device
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, self.n_heads, self.d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention(self.d_k, self.device)(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = nn.Linear(self.n_heads * self.d_v, self.d_model, device=self.device)(context)
        return nn.LayerNorm(self.d_model, device=self.device)(output + residual), attn # output: [batch_size x len_q x d_model]

Here is the PoswiseFeedForwardNet.

In [28]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(F.gelu(self.fc1(x)))

## 3.4 Putting them together

In [29]:
class BERT(nn.Module):
    def __init__(self, n_layers, n_heads, d_model, d_ff, d_k, n_segments, vocab_size, max_len, device):
        super(BERT, self).__init__()
        self.params = {'n_layers': n_layers, 'n_heads': n_heads, 'd_model': d_model,
                       'd_ff': d_ff, 'd_k': d_k, 'n_segments': n_segments,
                       'vocab_size': vocab_size, 'max_len': max_len}
        self.embedding = Embedding(vocab_size, max_len, n_segments, d_model, device)
        self.layers = nn.ModuleList([EncoderLayer(n_heads, d_model, d_ff, d_k, device) for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))
        self.device = device

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids, self.device)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]
        
        # 1. predict next sentence
        # it will be decided by first token(CLS)
        h_pooled   = self.activ(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_nsp = self.classifier(h_pooled) # [batch_size, 2]

        # 2. predict the masked token
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked  = self.norm(F.gelu(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_nsp
    
    def get_last_hidden_state(self, input_ids, segment_ids):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids, self.device)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)

        return output

## 4. Training

In [29]:
from tqdm.auto import tqdm

n_layers = 12    # number of Encoder of Encoder Layer
n_heads  = 12    # number of heads in Multi-Head Attention
d_model  = 768  # Embedding Size
d_ff = d_model * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

num_epoch = 1000
model = BERT(
    n_layers, 
    n_heads, 
    d_model, 
    d_ff, 
    d_k, 
    n_segments, 
    vocab_size, 
    max_len, 
    device
).to(device)  # Move model to GPU

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [30]:
batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

# Move inputs to GPU
input_ids = input_ids.to(device)
segment_ids = segment_ids.to(device)
masked_tokens = masked_tokens.to(device)
masked_pos = masked_pos.to(device)
isNext = isNext.to(device)

# Wrap the epoch loop with tqdm
for epoch in tqdm(range(num_epoch), desc="Training Epochs"):
    optimizer.zero_grad()
    logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)    
    #logits_lm: (bs, max_mask, vocab_size) ==> (6, 5, 34)
    #logits_nsp: (bs, yes/no) ==> (6, 2)

    #1. mlm loss
    #logits_lm.transpose: (bs, vocab_size, max_mask) vs. masked_tokens: (bs, max_mask)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    #2. nsp loss
    #logits_nsp: (bs, 2) vs. isNext: (bs, )
    loss_nsp = criterion(logits_nsp, isNext) # for sentence classification
    
    #3. combine loss
    loss = loss_lm + loss_nsp
    if epoch % 100 == 0:
        print('Epoch:', '%02d' % (epoch), 'loss =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Training Epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch: 00 loss = 117.719650


Training Epochs:  10%|█         | 101/1000 [00:35<05:15,  2.85it/s]

Epoch: 100 loss = 7.738173


Training Epochs:  20%|██        | 201/1000 [01:10<04:42,  2.83it/s]

...

Training Epochs:  80%|████████  | 801/1000 [04:41<01:11,  2.79it/s]

Epoch: 800 loss = 3.705327


Training Epochs:  90%|█████████ | 901/1000 [05:17<00:35,  2.81it/s]

Epoch: 900 loss = 3.702112


Training Epochs: 100%|██████████| 1000/1000 [05:52<00:00,  2.84it/s]


In [32]:
# Save the model after training
torch.save(model.state_dict(), 'bert_model.pth')
print("Model saved to bert_model.pth")

Model saved to bert_model.pth


## 5. Inference

Since our dataset is very small, it won't work very well, but just for the sake of demonstration.

In [31]:
# Predict Masked Tokens and NSP
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[2]))

print([id2word[w.item()] for w in input_ids[0] if id2word[w.item()] != '[PAD]'])

# Move inputs to device
input_ids = input_ids.to(device)
segment_ids = segment_ids.to(device)
masked_tokens = masked_tokens.to(device)
masked_pos = masked_pos.to(device)
isNext = isNext.to(device)

# Get model predictions
logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)
#logits_lm:  (1, max_mask, vocab_size) ==> (1, 5, 34)
#logits_nsp: (1, yes/no) ==> (1, 2)

#predict masked tokens
#max the probability along the vocab dim (2), [1] is the indices of the max, and [0] is the first value'
# Predict masked tokens
logits_lm = logits_lm.data.cpu().max(2)[1][0].numpy()  # Get highest probability words

print('Masked Tokens (Ground Truth): ', [id2word[int(pos.item())] for pos in masked_tokens[0]])
print('Masked Tokens List (Ground Truth): ', [int(pos.item()) for pos in masked_tokens[0]])
print('Predicted Masked Tokens (Words): ', [id2word[int(pos)] for pos in logits_lm.tolist()])
print('Predicted Masked Tokens List: ', logits_lm.tolist())

# Predict NSP
logits_nsp = logits_nsp.cpu().data.max(1)[1].item()  # Convert to scalar

print('Ground Truth isNext:', bool(isNext.item()))
print('Predicted isNext:', bool(logits_nsp))


['[CLS]', 'i', 'remember', 'now', 'that', 'he', 'told', 'me', 'a', 'while', 'ago', 'that', 'the', 'futons', '[MASK]', 'been', 'regularly', 'bugging', 'him', 'about', 'the', 'body', 'of', 'that', 'old', 'woman', 'found', 'in', 'the', 'construction', 'site', 'he', 'was', 'working', 'on', 'at', 'the', 'beginning', 'of', 'the', 'year', 'asking', 'him', 'the', 'same', 'questions', 'over', 'and', 'over', 'he', 'said', 'he', 'gives', 'the', 'same', 'answers', 'over', 'and', 'over', 'but', 'confessed', 'that', 'it', 'felt', 'like', 'they', 'suspected', 'he', 'either', 'had', 'something', 'to', 'do', 'with', 'the', 'ladys', 'death', 'or', 'knew', '[MASK]', 'who', 'had', '[SEP]', 'andre', 'hung', 'up', 'the', 'phone', 'but', 'he', 'didnt', 'turn', 'back', 'to', 'her', 'instead', 'he', 'sat', 'up', 'staring', 'at', 'the', 'cell', 'phone', 'in', 'his', 'hands', 'as', 'if', 'he', 'couldnt', 'quite', 'figure', 'what', 'to', 'do', 'with', 'it', 'stacey', 'sat', 'up', 'too', 'moving', 'to', 'put', 'he

Trying a bigger dataset should be able to see the difference.

## Task 2: Sentence Embedding with Sentence BERT

### Loading and Preprocessing SNLI dataset

In [30]:
#loading SNLI datasets

from datasets import load_dataset, Dataset

LargeDataset = load_dataset("snli")

#randomly sampling 50K rows from the train split
dataset = LargeDataset["train"].shuffle(seed=42).select(range(50000))

print(dataset[:5])

{'premise': ['A historian and his friend digging in the mines to look for more fossils for study.', 'Boy getting helped onto a merry-go-round.', 'A man making a contemplative pose in a laundry room.', 'A foreign man is preparing an ethnic dish.', 'Two young blond girls are eating with chopsticks.'], 'hypothesis': ['the historian is digging with his friend for study.', 'A boy is riding a donkey.', 'A man is outside on the patio.', 'A foreign man is preparing an ethnic dish, filled with colorful vegetables and fruit.', 'Some girls are eating Chinese food.'], 'label': [1, 2, 2, 1, 1]}


In [31]:
dataset

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 50000
})

In [32]:
#Lowercase
premise = [x.strip().lower() for x in dataset["premise"] if isinstance(x, str)]
hypothesis = [x.strip().lower() for x in dataset["hypothesis"] if isinstance(x, str)]

In [33]:
premise = [re.sub(r"[^\w\s]", '', x) for x in premise]  # Remove symbols
hypothesis = [re.sub(r"[^\w\s]", '', x) for x in hypothesis]

In [34]:
#building Vocab

word_list = list(set(" ".join(premise + hypothesis).split()))

#initializing word2id with special tokens
word2id_SBERT = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

for i, w in enumerate(word_list):
    word2id_SBERT[w] = i + 4  #starting indexing after special tokens

In [35]:
vocab_size = len(word2id_SBERT)

tokenized_premise = []
tokenized_hypothesis = []

for sentence in tqdm(premise, desc="Tokenizing Premises"):
    tokenized_premise.append([word2id_SBERT.get(word, word2id_SBERT["[PAD]"]) for word in sentence.split()])

Tokenizing Premises: 100%|██████████| 50000/50000 [00:00<00:00, 178581.94it/s]


In [36]:
for sentence in tqdm(hypothesis, desc="Tokenizing Hypotheses"):
    tokenized_hypothesis.append([word2id_SBERT.get(word, word2id_SBERT["[PAD]"]) for word in sentence.split()])

Tokenizing Hypotheses: 100%|██████████| 50000/50000 [00:00<00:00, 115287.42it/s]


In [37]:
for i in range(len(tokenized_premise)):
    tokenized_premise[i] = tokenized_premise[i][:128] + [word2id_SBERT["[PAD]"]] * (128 - min(len(tokenized_premise[i]), 128))
    tokenized_hypothesis[i] = tokenized_hypothesis[i][:128] + [word2id_SBERT["[PAD]"]] * (128 - min(len(tokenized_hypothesis[i]), 128))  

In [38]:
dataset_tokenized = dataset.add_column("input_ids1", tokenized_premise)   
dataset_tokenized = dataset_tokenized.add_column("input_ids2", tokenized_hypothesis)  #adding tockenized
dataset_tokenized = dataset_tokenized.remove_columns(["premise", "hypothesis"])   #removing untockenozed

In [39]:
dataset_tokenized

Dataset({
    features: ['label', 'input_ids1', 'input_ids2'],
    num_rows: 50000
})

### Dataloader

In [43]:

full_dataset = SNLIDataset(dataset_tokenized)

train_size = int(0.9 * len(full_dataset))
test_size = len(full_dataset) - train_size

train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False) 

### Trainning 

In [44]:
#Loading saved model from Task1
model = BERT(n_layers, n_heads, d_model, d_ff, d_k, n_segments, 114008, max_len, device).to(device)

In [ ]:
bert_model_path = "bert_model.pth"
model.load_state_dict(torch.load(bert_model_path, map_location=device))
model.eval() #Ensuring BERT embeddings stay the same while training SBERT

In [75]:
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}, requires_grad={param.requires_grad}")
    break  

decoder_bias: torch.Size([114008]), requires_grad=True


In [76]:
#siamese network for Sentence-BERT (SBERT)

#Implementing the SBERT class

class SBERT(nn.Module):
    def __init__(self, bert):
        super(SBERT, self).__init__()
        self.bert = bert         #use pre-trained BERT
        self.pooling = nn.AdaptiveAvgPool1d(1)   #average pooling for sentence embeddings
        self.fc = nn.Linear(3 * d_model, 3)     #softmax classifier for 3 classes
        

    def forward(self, input_ids1, segment_ids1, input_ids2, segment_ids2):
     
        #passing both sentences through the same BERT model as described in the paper
        output1 = self.bert.get_last_hidden_state(input_ids1, segment_ids1)
        output2 = self.bert.get_last_hidden_state(input_ids2, segment_ids2)

        #pooling
        u = self.pooling(output1.permute(0, 2, 1)).squeeze(-1)
        v = self.pooling(output2.permute(0, 2, 1)).squeeze(-1)

        # (u, v, |u - v|)
        combined = torch.cat([u, v, torch.abs(u - v)], dim=1)

        # Softmax classifier
        logits = self.fc(combined)
     
        return logits    

In [77]:
sbert_model = SBERT(model)
sbert_model.to(device)

SBERT(
  (bert): BERT(
    (embedding): Embedding(
      (tok_embed): Embedding(114008, 768)
      (pos_embed): Embedding(1000, 768)
      (seg_embed): Embedding(2, 768)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (layers): ModuleList(
      (0-11): 12 x EncoderLayer(
        (enc_self_attn): MultiHeadAttention(
          (W_Q): Linear(in_features=768, out_features=768, bias=True)
          (W_K): Linear(in_features=768, out_features=768, bias=True)
          (W_V): Linear(in_features=768, out_features=768, bias=True)
        )
        (pos_ffn): PoswiseFeedForwardNet(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (fc): Linear(in_features=768, out_features=768, bias=True)
    (activ): Tanh()
    (linear): Linear(in_features=768, out_features=768, bias=True)
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (cla

In [78]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(sbert_model.parameters(), lr=0.001)

In [ ]:
#Trainning

epochs = 1000

for epoch in tqdm(range(epochs), desc="Training Epochs"):
    epoch_loss = 0

    for batch in train_dataloader:
        optimizer.zero_grad()

      
        input_ids1 = batch["input_ids1"].to(device)
        input_ids2 = batch["input_ids2"].to(device)
        labels = batch["label"].to(device)

        segment_ids1 = torch.zeros_like(input_ids1, device=device)
        segment_ids2 = torch.zeros_like(input_ids2, device=device)

        logits = sbert_model(input_ids1, segment_ids1, input_ids2, segment_ids2)

        loss = criterion(logits, labels)
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch: {epoch:02d}, Loss: {loss.item():.6f}')



### Inference

In [ ]:
from sklearn.metrics import accuracy_score

sbert_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids1 = batch["input_ids1"].to(device)
        input_ids2 = batch["input_ids2"].to(device)
        labels = batch["label"].to(device)

        segment_ids1 = torch.zeros_like(input_ids1).to(device)
        segment_ids2 = torch.zeros_like(input_ids2).to(device)

        logits = sbert_model(input_ids1, segment_ids1, input_ids2, segment_ids2)
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(all_labels, all_preds)
print("Test Accuracy: {accuracy:.4f}")
